# Formula 1

In [40]:
import pandas as pd
import sqlite3
import requests
import csv

In [41]:
#create a connection
conn = sqlite3.connect('database.db')

# creating a cursor object
cur = conn.cursor()

In [43]:
#set the path
url = 'https://raw.githubusercontent.com/rahulakrish/formula_1/main/circuits.csv'

# Download the file
response = requests.get(url)

In [45]:
# Open the downloaded file for reading
with open('circuits.csv', 'w') as f:
    f.write(response.text)

# Read the data from the downloaded file
with open('circuits.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

In [48]:
#create a table
cur.execute('''CREATE TABLE circuit_info_new (
   circuit_id INT,
   circuitref VARCHAR,
   name VARCHAR,
   location VARCHAR,
   country VARCHAR,
   lat INT,
   lng INT,
   alt INT,
   url VARCHAR)''')

In [50]:
#insert data into the table
for row in data:
   cur.execute("INSERT INTO circuit_info_new (circuit_id, circuitref,name, location, country, lat, lng, alt, url) values (?, ?, ?, ?, ?, ?, ?, ?, ?)", row)

# committing changes
conn.commit()

In [53]:
cur.execute('''SELECT * FROM circuit_info_new LIMIT 2''').fetchall()

[('circuitId',
  'circuitRef',
  'name',
  'location',
  'country',
  'lat',
  'lng',
  'alt',
  'url'),
 (1,
  'albert_park',
  'Albert Park Grand Prix Circuit',
  'Melbourne',
  'Australia',
  -37.8497,
  144.968,
  10,
  'http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit')]